In [68]:
import cv2
import numpy as np

def get_minAreaRect(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray)
    thresh = cv2.threshold(gray, 0, 255,
        cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    coords = np.column_stack(np.where(thresh > 0))
    return cv2.minAreaRect(coords)


def rotate_bound(image, angle):
    # 获取宽高
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    # 提取旋转矩阵 sin cos
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # 计算图像的新边界尺寸
    nW = int((h * sin) + (w * cos))
    #     nH = int((h * cos) + (w * sin))
    nH = h

    # 调整旋转矩阵
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    return cv2.warpAffine(image, M, (nW, nH), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

In [69]:
img = cv2.imread('circle2.png')
cv2.imshow('img', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [70]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imshow('gray', gray)
cv2.waitKey()
cv2.destroyAllWindows()
print(img.shape)

(639, 1094, 3)


In [71]:
circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 100, param1=100, param2=30, minRadius=5, maxRadius=300)
print(type(circles))
print(len(circles[0]))
print(circles[0][1][2])
radius = []
for i in range(len(circles[0])):
    radius += [circles[0][i][2]]
    print(radius)
if len(circles[0]) > 3:
    radius.remove(min(radius))
    radius.remove(max(radius))
    radius_mean = int(np.mean(radius))
    print(radius_mean)
    # 筛选后的圆
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 100, param1=100, param2=30, minRadius=(radius_mean-10),
                               maxRadius=(radius_mean+10))
    print(circles)
    print(len(circles[0]))

<class 'numpy.ndarray'>
4
70.9
[69.5]
[69.5, 70.9]
[69.5, 70.9, 70.8]
[69.5, 70.9, 70.8, 23.4]
70
[[[232.5 479.5  69.5]
  [794.5 517.5  69.5]
  [882.5 165.5  70.8]]]
3


In [72]:
# 根据检测到圆的信息，画出每一个圆
for circle in circles[0]:
    # 圆的基本信息
    print("radius:", circle[2])
    # 坐标行列
    x = int(circle[0])
    y = int(circle[1])
    # 半径
    r = int(circle[2])
    circle_img = img[int(y-r/2):int(y+r/2), int(x-r/2):int(x+r/2)]
    cv2.imshow("corp", circle_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    angle = get_minAreaRect(circle_img)[-1]
    rotated = rotate_bound(circle_img, angle)
    cv2.imshow("rotate", rotated)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    rotated_gray = cv2.cvtColor(rotated, cv2.COLOR_BGR2GRAY)
    ret, binary = cv2.threshold(rotated_gray, 50, 255, cv2.THRESH_BINARY_INV)
    cv2.imshow("binary", binary)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    print(len(contours))
    cv2.drawContours(rotated, contours, -1, (0, 255, 0), 1)
    cv2.imshow("rotate", rotated)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    text = "x:" + str(x) + " y: " + str(y) + " R:" + str(r)
    # 在原图用指定颜色标记出圆的位置
    img = cv2.circle(img, (x, y), r, (0, 255, 0), -1)
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 2.0, (0, 0, 255), 2)

# 显示新图像
cv2.imshow('res', img)

# 按任意键退出
cv2.waitKey(0)
cv2.destroyAllWindows()

radius: 69.5
1
radius: 69.5
0
radius: 70.8
0
